In [1]:
!pip install numpy==1.19.5
!pip install --quiet lucid==0.2.3

%tensorflow_version 1.x  #required after tf 2.0 released
import numpy as np
import tensorflow as tf

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform
import operator
import pdb

'''
https://github.com/tensorflow/lucid/blob/master/lucid/modelzoo/vision_models.py
https://github.com/tensorflow/lucid/blob/master/lucid/modelzoo/other_models/InceptionV1.py
'''

googlenet = models.InceptionV1()
googlenet.load_graphdef()

# googlenet.model_path

# models.InceptionV1.layers
# models.InceptionV1().layers

     |████████████████████████████████| 14.8 MB 8.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 40 kB 3.2 MB/s 
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  #required after tf 2.0 released`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [12]:
# Dissect how each var in googlenet_semantic_dict() works

from lucid.misc.io import show, load
from lucid.misc.io.showing import _image_url

def googlenet_semantic_dict(layer, img_url):
    img = load(img_url)
    
    # Compute the activations
    with tf.Graph().as_default(), tf.Session():
        t_input = tf.placeholder(tf.float32, [224, 224, 3]) #empty tensor of fixed size
        T = render.import_model(googlenet, t_input, t_input)
        print(T(layer))
        acts = T(layer).eval({t_input: img})[0]  #[0] b/c it's the only item in a list
        print(acts.shape)
        #print(acts[:,:,0].shape)
        #print(acts[:,:,0].size)  #activation values of semantic dict for 0th neuron of layer
        # ggg = [[{"n": n, "v": float(act_vec[n])} for n in np.argsort(-act_vec)[:4]] for act_vec in acts[0,:,:]]
        # print(len(ggg[0]))
        # print(ggg[0])
        #print(len([[[{"n": n, "v": float(act_vec[n])} for n in np.argsort(-act_vec)[:4]] for act_vec in act_slice] for act_slice in acts]))
        #print([[[{"n": n, "v": float(act_vec[n])} for n in np.argsort(-act_vec)[:4]] for act_vec in act_slice] for act_slice in acts])
                   
# #     vals = acts[:,:,0].flatten()
# #     top_neurons_of_cell = vals.argsort()[-3:][::-1]
# #     print(top_neurons_of_cell)

#         top_neurons_of_cell = []
#         for dt in ggg[0]:
#           top_neurons_of_cell.append(list(dt.values())[0])
    
#     for i in top_neurons_of_cell:
#       obj = objectives.channel("mixed4d", i)
#       print('channel '+str(i))
#       render.render_vis(googlenet, obj)

# googlenet_semantic_dict("mixed4d", "https://storage.googleapis.com/lucid-static/building-blocks/examples/dog_cat.png")
# googlenet_semantic_dict("mixed4d", "https://png.pngtree.com/png-clipart/20190604/original/pngtree-dog-pet-golden-retriever-png-image_820349.jpg")
googlenet_semantic_dict("mixed4d", 'dog2.jpg')

Tensor("import/mixed4d:0", shape=(1, ?, ?, 528), dtype=float32)
[[[[  0.          0.          4.735308  ...   0.          0.
     67.1486   ]
   [  0.          0.         28.506151  ...   0.          0.
     73.85073  ]
   [  6.3043647   0.         15.023551  ...   0.          0.
     56.155212 ]
   ...
   [  0.          0.          0.        ...   0.          0.
     63.10116  ]
   [  0.          0.          0.        ...   0.          0.
     56.932    ]
   [  0.          0.          0.        ...   0.          0.
     45.951    ]]

  [[  0.          0.         18.063503  ...   0.          0.
     84.20397  ]
   [  0.          0.         40.747154  ...   0.          0.
     78.634865 ]
   [  0.          0.         25.654572  ...   0.          0.
     69.32244  ]
   ...
   [  0.          0.          5.548671  ...   0.          0.
     79.05948  ]
   [  0.          0.          0.        ...   0.          0.
     76.29701  ]
   [  0.          0.          0.        ...   0.          0.
 

In [4]:
from google.colab import files
uploaded = files.upload()

Saving dog2.jpg to dog2.jpg


In [ ]:
from lucid.misc.io import show, load
from lucid.misc.io.showing import _image_url

def output_top_neurons(layer, img_url, num_output):
    #num_output : top N neurons to output

    img = load(img_url)
    img = np.resize(img,(224,224,3))
    
    # Compute the activations
    with tf.Graph().as_default(), tf.Session():
        t_input = tf.placeholder(tf.float32, [224, 224, 3]) #empty tensor of fixed size
        T = render.import_model(googlenet, t_input, t_input)
        acts = T(layer).eval({t_input: img})[0]

        top_actv = [[[{"n": n, "v": float(act_vec[n])} for n in np.argsort(-act_vec)[:num_output]] \
                     for act_vec in act_slice] for act_slice in acts]
        flat_list = [item for sublist in top_actv for item in sublist]
    
    top_neurons = []  #a list of a list of top N neurons (not their actv vals) for each of 196 patches
    for i in range(196):
      top_neurons_in_patch = []
      for dt in flat_list[i]:
          top_neurons_in_patch.append(list(dt.values())[0])
      top_neurons_in_patch.sort()
      top_neurons.append(top_neurons_in_patch)
    return top_neurons
  
# exhaustively compare every pair of patches
def map_semantic_dict(img1_neurons, img2_neurons):
  output_dict = {}
  for i, lst1 in enumerate(img1_neurons): #i: a patch in img1
    for j, lst2 in enumerate(img2_neurons):
      common_elem = set(lst1) - (set(lst1) - set(lst2))  #neurons in both patches
      if i in output_dict:
        prev_best = set(lst1) - (set(lst1) - set(img2_neurons[output_dict[i]]))
        if len(common_elem) > len(prev_best):  #this patch j has more matches with patch i than prev patch
          output_dict[i] = j  #map patch i to patch j 
      else:
        output_dict[i] = j  #if same or less number of common neurons, retain first match
  return output_dict

dog1_url = "https://i.natgeofe.com/n/4f5aaece-3300-41a4-b2a8-ed2708a0a27c/domestic-dog_thumb_4x3.jpg"

top_neurons_img1 = output_top_neurons("mixed4d", dog1_url, 4)
top_neurons_img2 = output_top_neurons("mixed4d", 'dog2.jpg', 4)
top_neurons_img3 = output_top_neurons("mixed4d", 'dog3.jpg', 4)

mappings = map_semantic_dict(top_neurons_img1, top_neurons_img2)  #b/w patches in the 2 images
mappings_2 = map_semantic_dict(top_neurons_img1, top_neurons_img3)

In [ ]:
# Get feature viz for the top 4 neurons that activate the most at the nose patch

for i in top_neurons_img1[90]:  #nose
  obj = objectives.channel("mixed4d", i)
  print('channel '+str(i))
  render.render_vis(googlenet, obj)

In [ ]:
# Mappings maps each patch in image1 to a patch in image2 w/ the most common highest neuron actvs
# mappings[90] gets the patch in image2 that is mapped to patch 90 in image1
mappings[90]

117

In [ ]:
# Check which neurons these two mapped patches share
print(top_neurons_img1[90]) # The top 4 neurons that activate for patch 90 in image1
top_neurons_img2[mappings[90]] #use print() in notebook else only the last line is displayed

[24, 286, 408, 416]


[82, 380, 408, 424]

In [ ]:
mappings_2[90]

103

In [ ]:
# Feature viz the top 4 neurons in the patch in dog3 that maps to patch 90 in dog2

for i in top_neurons_img2[mappings[90]]:
  obj = objectives.channel("mixed4d", i)
  print('channel '+str(i))
  render.render_vis(googlenet, obj)

In [ ]:
# Feature viz top neurons for both patch 61 and its mapped patch

for i in top_neurons_img1[61]:  #nose
  obj = objectives.channel("mixed4d", i)
  print('channel '+str(i))
  render.render_vis(googlenet, obj)
  
for i in top_neurons_img2[mappings[61]]:
  obj = objectives.channel("mixed4d", i)
  print('channel '+str(i))
  render.render_vis(googlenet, obj)

In [ ]:
# Get the intersection of the common top neurons b/w patch 61 and its mapped patch
# Use top 10 neurons

top_10_img1_patch61 = output_top_neurons("mixed4d", 'dog2.jpg', 10)[61]
top_10_img2_patch60 = output_top_neurons("mixed4d", 'dog3.jpg', 10)[mappings[61]]
set(top_10_img1_patch61) - ( set(top_10_img1_patch61) - set(top_10_img2_patch60) )

{49, 404, 460}

In [ ]:
# Instead of mapping a patch to one other patch, map it to the top N other patches

def rank_patch_matches(img1_neurons, img2_neurons, num_patches):
    patch_mapping = {}  #patch in img1 : ranked list of patch tuples (patch in img2, # common neurons, set of neurons in common)
    for patch_1, neurons_1 in enumerate(img1_neurons): #a patch in img1, its ranking of neuron activations
        ranked_list = []
        for patch_2, neurons_2 in enumerate(img2_neurons):
            patch_2_tuple = [patch_2]
            common_elem = set(neurons_1).intersection(set(neurons_2))  #common neurons in both patches
            patch_2_tuple.append(len(common_elem))
            patch_2_tuple.append(common_elem)
            ranked_list.append(tuple(patch_2_tuple))
        ranked_list.sort(key = lambda x: x[1]) # sort tuples by 2nd element of tuple
        patch_mapping[patch_1] = ranked_list[-num_patches::]
    return patch_mapping
        
img1_neurons = output_top_neurons("mixed4d", 'dog2.jpg', 4)
img2_neurons = output_top_neurons("mixed4d", 'dog3.jpg', 4)
patch_mapping = rank_patch_matches(img1_neurons, img2_neurons, 10)

patch_mapping[91]

[(103, 1, {517}),
 (105, 1, {517}),
 (106, 1, {517}),
 (107, 1, {517}),
 (149, 1, {517}),
 (164, 1, {517}),
 (190, 1, {517}),
 (21, 2, {488, 517}),
 (34, 2, {488, 517}),
 (35, 2, {488, 517})]

In [ ]:
# # Debug by finding out values of var inside fn

# img1_neurons = output_top_neurons("mixed4d", 'dog2.jpg', 4)
# img2_neurons = output_top_neurons("mixed4d", 'dog3.jpg', 4)
# num_patches = 10

# patch_mapping = {}  #patch in img1 : ranked list of patch tuples (patch in img2, # common neurons, set of neurons in common)
# for patch_1, neurons_1 in enumerate(img1_neurons): #a patch in img1, its ranking of neuron activations
#   # patch_ranking = {}  # patch in img2 : # common neurons w/ img1
#   # patch_ranking_2 = {}  # patch in img2 : set of neurons in common
#   ranked_list = []
#   for patch_2, neurons_2 in enumerate(img2_neurons):
#     patch_2_tuple = [patch_2]
#     common_elem = set(neurons_1).intersection(set(neurons_2))  #common neurons in both patches
#     # patch_ranking[patch_2] = len(common_elem)
#     # patch_ranking_2[patch_2] = common_elem
#     patch_2_tuple.append(len(common_elem))
#     patch_2_tuple.append(common_elem)
#     ranked_list.append(patch_2_tuple)
#   # sorted_patch_ranking = sorted(patch_ranking.items(), key=operator.itemgetter(1))
#   ranked_list.sort(key = lambda x: x[1]) 
#   patch_mapping[patch_1] = ranked_list[-num_patches::]

#   # patch_mapping_2[i] = patch_ranking_2
# # patch_mapping, patch_mapping_2
# patch_mapping[91]

In [ ]:
#give weight to ranking; binary in/out and activation values

In [ ]:
#function to label patch numbers, overlaying grid over image

In [ ]:
#run on 10 different images. Occlusion experiments.

In [ ]:
#gets aggregate stats such as 'avg # of common neurons b/w 2 mapped patches'



---

